In [18]:
# insert train_split data as Pandas Dataframe Object and remove the df* variable

train_df = pd.read_csv(body)
train_df.head()



(354951, 44)

In [20]:
train_df.shape

(532428, 45)

In [22]:
from sklearn import preprocessing
import numpy as np

def prep_data(WORKING_DF):
    encoded = pd.DataFrame()
    
    WORKING_DF.drop(columns=['batch_enrolled', 'desc', 'zip_code'], axis=1)
    # Mapping and encoding emp_length values
    scale_mapper = {np.nan:0, '< 1 year':1, '1 year':2, '2 years':3, '3 years':4, '4 years':5, '5 years':6, '6 years':7, '7 years':8, '8 years':9, '9 years':10, '10+ years':11}
    encoded['emp_length_encoded'] = WORKING_DF['emp_length'].replace(scale_mapper)

    # Encoding remaining ordinal variables
    grouped = WORKING_DF[['delinq_2yrs', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'pub_rec', 'total_acc', 'open_acc', 'collections_12_mths_ex_med', 'acc_now_delinq', 'last_week_pay']]
    grouped = grouped.apply(preprocessing.LabelEncoder().fit_transform)
    encoded = pd.concat([encoded, grouped], axis=1)


    # One-hot encode nominal variables
    grouped = WORKING_DF[['term', 'grade', 'sub_grade', 'home_ownership', 'verification_status', 'pymnt_plan', 'purpose', 'addr_state', 'initial_list_status', 'application_type', 'verification_status_joint', 'member_id']]
    grouped = pd.get_dummies(grouped)
    encoded = pd.concat([encoded, grouped], axis=1)

    # Append float columns to encoded df
    grouped = WORKING_DF[['annual_inc', 'dti', 'revol_bal', 'revol_util', 'total_acc', 'total_rec_int', 'total_rec_late_fee', 'recoveries',
                          'collection_recovery_fee', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim', 'open_acc', 'mths_since_last_major_derog']]

    # Fill NaN values with mean of each column
    fill_NaN = preprocessing.Imputer(missing_values=np.nan, strategy='mean', axis=0)
    imputed_df = pd.DataFrame(fill_NaN.fit_transform(grouped))
    imputed_df.columns = grouped.columns
    imputed_df.index = grouped.index

    encoded = pd.concat([encoded, imputed_df], axis=1)
    return encoded

In [23]:
encoded = prep_data(train_df)
encoded_test_df = prep_data(test_df)
encoded = encoded.drop(columns=['home_ownership_ANY'], axis=1)

In [24]:
encoded_test_df.head()

,emp_length_encoded,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,pub_rec,total_acc,open_acc,collections_12_mths_ex_med,acc_now_delinq,...,total_acc,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,open_acc,mths_since_last_major_derog
0,5,1,1,20,57397,0,52,16,0,0,...,53.0,3915.61,0.0,0.0,0.0,0.0,85230.0,45700.0,16.0,44.079923
1,6,0,0,101777,191677,0,62,8,0,0,...,63.0,1495.06,0.0,0.0,0.0,0.0,444991.0,21400.0,8.0,44.079923
2,11,1,0,23,115,1,19,11,0,0,...,20.0,2096.21,0.0,0.0,0.0,0.0,105737.0,16300.0,11.0,26.000000
3,11,0,0,101778,191676,0,25,21,0,0,...,26.0,1756.31,0.0,0.0,0.0,0.0,287022.0,72400.0,21.0,44.079923
4,6,0,0,101779,191675,0,35,16,0,0,...,36.0,172.21,0.0,0.0,0.0,0.0,234278.0,26700.0,16.0,44.079923


In [25]:
from sklearn.model_selection import train_test_split

X = encoded
y = train_df['loan_status']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [78]:
from sklearn.linear_model import LinearRegression

lr_clf = LinearRegression()
lr_clf.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [80]:
lr_clf.predict([X_test.iloc[1,:]])

array([ 0.72407454])

In [ ]:
lr_clf.predict(X_test)

In [81]:
lr_clf.score(X_test, y_test)

0.31840236226843988